In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Configuration
CASES = ["A", "B", "C", "D"]

# Notebook location
BASE_DIR = os.getcwd()
LOAD_OUTPUT_PATH = os.path.abspath(os.path.join(BASE_DIR, '../../../Load Anywhere Output'))

# Load CSV files from a folder
def load_timestep_data(folder_path):
    files = [f for f in os.listdir(folder_path) if f.endswith(".csv")]
    all_data = []
    for file in files:
        df = pd.read_csv(os.path.join(folder_path, file))
        all_data.append(df[["Timestep", "Jeep Spatial Mean Speed", "Truck Spatial Mean Speed",
                            "Vehicle Spatial Mean Speed", "Actual Density"]])
    return all_data

# Compute average and std dev of speed for time series
def compute_time_series_statistics(all_data):
    df_concat = pd.concat(all_data, axis=0, keys=range(len(all_data)))
    mean_speed = df_concat.groupby("Timestep").mean()
    std_speed = df_concat.groupby("Timestep").std()
    return mean_speed, std_speed

# Compute variance reduction by increasing trial count
def compute_variance_reduction(all_data):
    num_trials = len(all_data)
    jeep_vars, truck_vars, vehicle_vars = [], [], []
    for n in range(10, num_trials + 1, 10):
        subset = all_data[:n]
        df_concat = pd.concat(subset, axis=0, keys=range(n))
        mean = df_concat.groupby("Timestep").mean()
        jeep_vars.append(mean["Jeep Spatial Mean Speed"].var())
        truck_vars.append(mean["Truck Spatial Mean Speed"].var())
        vehicle_vars.append(mean["Vehicle Spatial Mean Speed"].var())
    return jeep_vars, truck_vars, vehicle_vars

# Plot mean speed time series
def plot_time_series_per_key(case, key, data, output_root):
    mean_speed, std_speed = compute_time_series_statistics(data)
    output_dir = os.path.join(output_root, f"plots_by_case_{key}", f"Case_{case}")
    os.makedirs(output_dir, exist_ok=True)

    speed_types = {
        "Jeep Spatial Mean Speed": "Jeep",
        "Truck Spatial Mean Speed": "Truck",
        "Vehicle Spatial Mean Speed": "Vehicle"
    }

    for speed_key, label in speed_types.items():
        if speed_key in mean_speed.columns:
            plt.figure(figsize=(12, 6))
            plt.plot(mean_speed.index, mean_speed[speed_key], label="Mean")
            plt.fill_between(mean_speed.index,
                             mean_speed[speed_key] - std_speed[speed_key],
                             mean_speed[speed_key] + std_speed[speed_key],
                             alpha=0.2)
            plt.xlabel("time", fontsize=30)
            plt.ylabel(r"$<v>_{spatial}$", fontsize=30)
            plt.xticks(fontsize=15)
            plt.yticks(fontsize=15)
            plt.title(f"Case {case} | {label} | {key}", fontsize=14)
            plt.grid(True)
            plt.tight_layout()
            filename = f"Case_{case}_{key}_{label}_Spatial_Mean_Speed.png"
            plt.savefig(os.path.join(output_dir, filename), bbox_inches="tight")
            plt.close()

# Plot variance reduction
def plot_variance_reduction_per_key(case, key, data, output_root):
    if len(data) < 10:
        return

    jeep_vars, truck_vars, vehicle_vars = compute_variance_reduction(data)
    x_vals = list(range(0, len(jeep_vars) * 10 + 1, 10))
    jeep_vars = [jeep_vars[0]] + jeep_vars
    truck_vars = [truck_vars[0]] + truck_vars
    vehicle_vars = [vehicle_vars[0]] + vehicle_vars

    output_dir = os.path.join(output_root, f"plots_by_case_{key}", f"Case_{case}")
    os.makedirs(output_dir, exist_ok=True)

    for vars_list, label in zip([jeep_vars, truck_vars, vehicle_vars], ["Jeep", "Truck", "Vehicle"]):
        plt.figure(figsize=(12, 6))
        plt.plot(x_vals, vars_list, marker='o')
        plt.xlabel("number of trials", fontsize=30)
        plt.ylabel(r"variance of $<v>_{spatial}$", fontsize=30)
        plt.xticks(fontsize=15)
        plt.yticks(fontsize=15)
        plt.title(f"Case {case} | {label} | {key}", fontsize=14)
        plt.grid(True)
        plt.tight_layout()
        filename = f"Case_{case}_{key}_{label}_VarianceReduction.png"
        plt.savefig(os.path.join(output_dir, filename), bbox_inches="tight")
        plt.close()

# Main loop
for case in CASES:
    print(f"Processing Case {case}...")
    all_case_data = {}
    case_base = os.path.join(LOAD_OUTPUT_PATH, f"Case_{case}_StopToStop_20")
    if not os.path.exists(case_base):
        print(f"❌ Path not found: {case_base}")
        continue

    for kappa_folder in sorted(os.listdir(case_base)):
        if not kappa_folder.startswith("Kappa_"):
            continue
        kappa_val = kappa_folder.replace("Kappa_", "")
        kappa_path = os.path.join(case_base, kappa_folder)

        for density_folder in sorted(os.listdir(kappa_path)):
            if not density_folder.startswith("Density_"):
                continue
            density_val = density_folder.replace("Density_", "")
            density_path = os.path.join(kappa_path, density_folder)

            for rate_folder in sorted(os.listdir(density_path)):
                if not rate_folder.startswith("PassengerRate_"):
                    continue
                rate_val = rate_folder.replace("PassengerRate_", "")
                timestep_path = os.path.join(density_path, rate_folder, "TimestepSummary")

                if os.path.exists(timestep_path):
                    key = f"K{kappa_val}_R{rate_val}_D{density_val}"
                    if key not in all_case_data:
                        all_case_data[key] = []
                    all_case_data[key].extend(load_timestep_data(timestep_path))
                else:
                    print(f"⚠️ Missing: {timestep_path}")

    for key, data in all_case_data.items():
        plot_time_series_per_key(case, key, data, BASE_DIR)
        plot_variance_reduction_per_key(case, key, data, BASE_DIR)

print("✅ All plots saved.")
